# Random Shaking Volume Denoising (RSVD) using opticalflow3d (Corbel)

In [ ]:
local_debug = True

In [ ]:
jupyter_server = True

In [ ]:
vol_filename = "Corbel2301_block2_June2019_crop_ali_crop.mrc"

In [ ]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    print("Running in Colab")
    !pip install cupy-cuda12x # Ojo, posiblemente también hay que instalarlo en cualquier máquina y borrar cupy-cuda113
    !pip install opticalflow3D
    !apt install libcudart11.0
    !apt install libcublas11
    !apt install libcufft10
    !apt install libcusparse11
    !apt install libnvrtc11.2
    #from google.colab import drive
    #drive.mount('/content/drive')
    #!cp drive/Shareddrives/TomogramDenoising/tomograms/{vol_file}
else:
    print("Running in locahost")
    #!cp ~/Downloads/{vol_name}.tif .

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import logging
import mrcfile
warnings.filterwarnings("ignore", category=NumbaPerformanceWarning)

In [ ]:
if local_debug:
    !ln -sf ../../motion_estimation/src/motion_estimation .
else:
    !pip install "denoising @ git+https://github.com/vicente-gonzalez-ruiz/motion_estimation"

In [ ]:
if local_debug:
    !ln -sf denoising/volume/RSVD.py .
else:
    !pip install "denoising @ git+https://github.com/vicente-gonzalez-ruiz/denoising"
import RSVD

In [ ]:
if local_debug:
    !ln -sf ../../information_theory/src/information_theory/ .
else:
    !pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"
import information_theory  # pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"

In [ ]:
#import skimage.io

In [ ]:
#import RSIVD

In [ ]:
from collections import namedtuple
Args = namedtuple("args", ["input", "output"])
fn, fe = vol_filename.split(".")
args = Args(vol_filename , fn + "_denoised." + fe)
print(args)

In [ ]:
%%bash -s "$args.input"
set -x
OUTPUT_FILENAME=$1
#rm -f $OUTPUT_FILENAME
if test ! -f $OUTPUT_FILENAME ; then
    FILEID="1Uqa6ywi8bllhyHxrODD5yjuesUkNO3O0" # https://drive.google.com/file/d/1Uqa6ywi8bllhyHxrODD5yjuesUkNO3O0/view?usp=drive_link
    #wget --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O $OUTPUT_FILENAME #2> /dev/null
    #wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=FILEID' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=FILEID" -O FILENAME && rm -rf /tmp/cookies.txt
    gdown https://drive.google.com/uc?id=$FILEID # pip install gdown
fi
set +x

In [ ]:
#noisy = opticalflow3D.helpers.load_image(args.input)
stack_MRC = mrcfile.open(args.input)
noisy = stack_MRC.data

In [ ]:
noisy = (255*(noisy - np.min(noisy))/(np.max(noisy) - np.min(noisy))).astype(np.uint8)

In [ ]:
noisy.shape

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(16, 16))
axs.imshow(noisy[:, ::-1, :][50], cmap="gray")
axs.set_title(f"Noisy")
fig.tight_layout()
plt.show()

In [ ]:
#noisy = noisy[50:, 200:, 200:]

In [ ]:
block_size = (noisy.shape[0], noisy.shape[1], noisy.shape[2])

In [ ]:
#farneback = opticalflow3D.Farneback3D(iters=5, num_levels=3, scale=0.5, spatial_size=7, presmoothing=3, filter_type="gaussian", filter_size=7,); RS_sigma = 1.0
#farneback = opticalflow3D.Farneback3D(iters=5, num_levels=2, scale=0.5, spatial_size=5, sigma_k=1.0, filter_type="gaussian", filter_size=9, presmoothing=None, device_id=0); RS_sigma = 1.25; N_iters=100
denoiser = RSVD.Random_Shaking_Denoising(logging_level=logging.DEBUG)
#denoiser = Random_Shaking_Denoising(logging_level=logging.DEBUG)

In [ ]:
"""Farneback3D class used to instantiate the algorithm with its parameters.

Args:
    iters (int): number of iterations. Defaults to 5
    num_levels (int): number of pyramid levels. Defaults to 5
    scale (float): Scaling factor used to generate the pyramid levels. Defaults to 0.5
    spatial_size (int): size of the support used in the calculation of the standard deviation of the Gaussian
        applicability. Defaults to 9.
    sigma_k (float): scaling factor used to calculate the standard deviation of the Gaussian applicability. The
        formula to calculate sigma is sigma_k*(spatial_size - 1). Defaults to 0.15.
    filter_type (str): Defines the type of filter used to average the calculated matrices. Defaults to "box"
    filter_size (int): Size of the filter used to average the matrices. Defaults to 21
    presmoothing (int): Standard deviation used to perform Gaussian smoothing of the images. Defaults to None
    device_id (int): Device id of the GPU. Defaults to 0
"""
RS_sigma = 5.0
N_iters = 100
denoised = denoiser.filter_volume(noisy, std_dev=RS_sigma, N_iters=N_iters, block_size=block_size)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
axs[0].imshow(noisy[25], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised[25], cmap="gray")
axs[1].set_title(f"Denoised (DQI={information_theory.information.compute_quality_index(noisy[16], denoised[16])})")
fig.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
axs[0].imshow(noisy[1][300:,300:], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised[1][300:,300:], cmap="gray")
axs[1].set_title(f"Denoised (DQI={information_theory.information.compute_quality_index(noisy[16][300:,300:], denoised[16][300:,300:])}")
fig.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
axs[0].imshow(noisy[:, 100], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised[:, 100], cmap="gray")
axs[1].set_title(f"Denoised (DQI={information_theory.information.compute_quality_index(noisy[:, 100], denoised[:, 100])})")
fig.tight_layout()
plt.show()

In [ ]:
figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
imgplot = plt.imshow(noisy[7][::-1, :], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("$\sigma_\mathrm{RS}=$"+f"{RS_sigma}")
plt.imshow(denoised[7][::-1, :], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(noisy[7][::-1, :] - denoised[7][::-1, :], cmap="gray")

In [ ]:
with mrcfile.new(f"{fn}_{RS_sigma}_{N_iters}.mrc", overwrite=True) as mrc:
            mrc.set_data(denoised.astype(np.float32))
            mrc.data
#skimage.io.imsave(f"{args.output}_{RS_sigma}_{N_iters}.tif", denoised, imagej=True)
f"{fn}_{RS_sigma}_{N_iters}.mrc"

In [ ]:
input()

In [ ]:
farneback = opticalflow3D.Farneback3D(iters=5,
                                      num_levels=3,
                                      scale=0.5,
                                      spatial_size=5,
                                      presmoothing=4,
                                      filter_type="box",
                                      filter_size=5,
                                     )

In [ ]:
RS_sigma = 1.0
denoised_vol = RSIVD.filter(farneback, block_size, noisy_vol, RS_sigma=RS_sigma, N_iters=25)

In [ ]:
figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
imgplot = plt.imshow(noisy_vol[75][::-1, :], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("$\sigma_\mathrm{RS}=$"+f"{RS_sigma}")
plt.imshow(denoised_vol[75][::-1, :], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(noisy_vol[75][::-1, :] - denoised_vol[75][::-1, :], cmap="gray")

In [ ]:
skimage.io.imsave(f"{vol_name}_denoised_{RS_sigma}.tif", denoised_vol, imagej=True)

In [ ]:
RS_sigma = 2.0
denoised_vol = RSIVD.filter(farneback, block_size, noisy_vol, RS_sigma=RS_sigma, N_iters=25)

In [ ]:
figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
imgplot = plt.imshow(noisy_vol[75][::-1, :], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("$\sigma_\mathrm{RS}=$"+f"{RS_sigma}")
plt.imshow(denoised_vol[75][::-1, :], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(noisy_vol[75][::-1, :] - denoised_vol[75][::-1, :], cmap="gray")